In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [8]:
BIT_CNT_EXT = 28

In [9]:
df = pd.read_csv("out.csv")
df.head()

,angle,sin,cos
0,0,0,10000000
1,2710,270f,fffffff
2,4e20,4e1f,fffffff
3,7530,752f,ffffffe
4,9c40,9c3f,ffffffd


In [10]:
def ToDouble(x):
    result = 0.0
    x = bin(int(x, 16))[2:].zfill(BIT_CNT_EXT+2)
    temp = 1.0
    for bit in x[1:]:
        if bit == '1':
            result += temp
        temp = temp/2
    if x[0] == '1':
        result -= 2
    return result

for col in df:
    df["raw_" + str(col)] = df[col]
    df[col] = df[col].apply(ToDouble)
df.head()

,angle,sin,cos,raw_angle,raw_sin,raw_cos
0,0.000000,0.000000,1.0,0,0,10000000
1,0.000037,0.000037,1.0,2710,270f,fffffff
2,0.000075,0.000075,1.0,4e20,4e1f,fffffff
3,0.000112,0.000112,1.0,7530,752f,ffffffe
4,0.000149,0.000149,1.0,9c40,9c3f,ffffffd


In [11]:
def errF_ULP(angle, val, fun):
    ULP = 2**-BIT_CNT_EXT
    temp = fun(angle)
    err = np.abs(val - temp)
    ulpErr = err/ULP
    return ulpErr

def errF(angle, val, fun):
    temp = fun(angle)
    err = np.abs(val - temp)
    return err

df['ErrSin'] = df.apply(lambda x : errF(x.angle, x.sin, np.sin), axis=1)
df['ErrCos'] = df.apply(lambda x : errF(x.angle, x.cos, np.cos), axis=1)
df['ErrSin[ULP]'] = df.apply(lambda x : errF_ULP(x.angle, x.sin, np.sin), axis=1)
df['ErrCos[ULP]'] = df.apply(lambda x : errF_ULP(x.angle, x.cos, np.cos), axis=1)
df.head()

,angle,sin,cos,raw_angle,raw_sin,raw_cos,ErrSin,ErrCos,ErrSin[ULP],ErrCos[ULP]
0,0.000000,0.000000,1.0,0,0,10000000,0.000000e+00,0.000000e+00,0.000000,0.000000
1,0.000037,0.000037,1.0,2710,270f,fffffff,3.725282e-09,3.031401e-09,0.999998,0.813735
2,0.000075,0.000075,1.0,4e20,4e1f,fffffff,3.725221e-09,9.497327e-10,0.999981,0.254942
3,0.000112,0.000112,1.0,7530,752f,ffffffe,3.725058e-09,1.205576e-09,0.999938,0.323619
4,0.000149,0.000149,1.0,9c40,9c3f,ffffffd,3.724739e-09,7.364065e-11,0.999852,0.019768


In [12]:
df[['ErrSin[ULP]', 'ErrCos[ULP]', 'ErrSin', 'ErrCos']].describe(include='all')

,ErrSin[ULP],ErrCos[ULP],ErrSin,ErrCos
count,42166.000000,42166.000000,4.216600e+04,4.216600e+04
mean,0.499999,0.499731,1.862641e-09,1.861641e-09
std,0.289466,0.289058,1.078346e-09,1.076824e-09
min,0.000000,0.000000,0.000000e+00,0.000000e+00
25%,0.248356,0.248904,9.251998e-10,9.272413e-10
50%,0.499202,0.498373,1.859671e-09,1.856586e-09
75%,0.752077,0.750560,2.801706e-09,2.796055e-09
max,0.999998,0.999980,3.725282e-09,3.725215e-09
